## Recon-all of Scott 10K

### Prelims: create error log files

In [59]:
%%bash

mkdir -p /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/

### 1. Create script of all T1 patients from ADNI

In [1]:
%%bash 

mydir=/rds/general/project/scott_data_adni/live/ADNI/ADNI_NIFTI/
output_file=/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_t1_paths.txt
>"$output_file"
find "$mydir" -type f -name "*.nii.gz" >> "$output_file"

wc -l<"${output_file}"

15733


### 2. Submit a recon-all job. Tinker around with CPUs memory and walltime for optimisation [ARCHIVED]

In [62]:
# %%file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh

# #!/bin/bash
# #PBS -l walltime=36:00:00
# #PBS -l select=1:ncpus=1:mem=20gb
# #PBS -J 1-10000
# #PBS -N reconall_scott_10k
# #PBS -o /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/
# #PBS -e /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/

# inpath=`head -n ${PBS_ARRAY_INDEX} /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_t1_paths.txt| tail -n 1`
# infile=`basename $inpath .nii.gz`;

# echo ${infile}

# outdir=/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k
# mkdir -p "$outdir"

# module load tools/prod
# module load FreeSurfer/7.4.1-centos8_x86_64
# module load FSL/6.0.5.1-foss-2021a
# module load MATLAB/2024b

# export JOB_NUM=$(echo ${PBS_JOBID} | cut -f 1 -d '.' | cut -f 1 -d '[')
# export NEW_TMPDIR="${EPHEMERAL}/${JOB_NUM}.${PBS_ARRAY_INDEX}"
# mkdir -p ${NEW_TMPDIR}
# export TMPDIR=${NEW_TMPDIR}
# export SUBJECTS_DIR="/rds/general/project/scott_data_adni/live/ADNI/ADNI_NIFTI/ADNI_allT1/"
# export FS_LICENSE="${HOME}/license.txt"

# cmd="recon-all -all -i ${inpath} -subjid ${infile} -sd ${outdir}"
# echo ${cmd}
# ${cmd}

# cd ${outdir}
# find . -print0 | while IFS= read -r -d '' path;
#   do touch "$path" ;
# done

In [63]:
# %%bash

# chmod -Rf 775 /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh
# qsub /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh

### Refresh timestamps script

In [4]:
%%file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/refresh_reconall_timestamps.sh

#!/bin/bash
#PBS -l walltime=06:00:00
#PBS -l select=1:ncpus=1:mem=1gb
#PBS -N refresh_reconall
#PBS -o /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/
#PBS -e /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/

outdir=/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k

for i in $(seq 1 1000); do
    echo ${i}
    cd ${outdir}
    find . -print0 | while IFS= read -r -d '' path;
      do touch "$path" ;
    done
done

Writing /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/refresh_reconall_timestamps.sh


In [3]:
%%bash

chmod -Rf 775 /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/refresh_reconall_timestamps.sh
qsub /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/refresh_reconall_timestamps.sh

1210637.pbs-7


#### This code block just lets me check how many successful recon-all jobs I have

In [13]:
import os
import shutil
import fnmatch
from tqdm import tqdm

path = '/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k'
 
subdirs = os.listdir(path)
counter = 0

for subdir in tqdm(subdirs, desc='Checking all subjects for completed recon-alls: '):
    log_path = os.path.join(path, subdir, 'scripts', 'recon-all.log')
    try:
        if os.path.exists(log_path):
            with open(log_path, 'r') as log:
                contents = log.read()
                lines = contents.splitlines()
                success = any(
                    fnmatch.fnmatch(line, "recon-all * finished without error at *")
                    for line in lines
                )
                if success:
                    counter +=1
    except DirectoryNotFoundError or FileNotFoundError:
        continue

print(counter)

Checking all subjects for completed recon-alls: 100%|██████████| 11744/11744 [06:39<00:00, 29.36it/s]

11177


### Cleaning out patients whose recon-alls errored out and rerunning them again

In [14]:
import os
import shutil
import fnmatch
from tqdm import tqdm

# Root path
path = '/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k'

dry_run = False 

subdirs = os.listdir(path)

for subdir in tqdm(subdirs, desc="Checking subjects"):
    log_path = os.path.join(path, subdir, 'scripts', 'recon-all.log')
    try:
        if os.path.exists(log_path):
            with open(log_path, 'r') as log:
                contents = log.read()
                lines = contents.splitlines()
                success = any(
                    fnmatch.fnmatch(line, "recon-all * finished without error at *")
                    for line in lines
                )
                if success:
                    continue
                else:
                    tqdm.write(f"{subdir}: failed {'(would remove)' if dry_run else '(removing)'}")
                    if not dry_run:
                        shutil.rmtree(os.path.join(path, subdir))
                        print(f"REMOVING {subdir}")
        else:
            tqdm.write(f"{subdir}: no log file {'(would remove)' if dry_run else '(removing)'}")
            if not dry_run:
                shutil.rmtree(os.path.join(path, subdir))
                print(f"REMOVING {subdir}")
    except NotADirectoryError:
        print(f'Not a directory: {os.path.join(path, subdir)}')
        try:
            os.remove(os.path.join(path, subdir))
        except Exception as e:
            print(e)
            continue
    except FileNotFoundError:
        print(f'Not a valid file path: {os.path.join(path, subdir)}')
        try:
            os.remove(os.path.join(path, subdir))
        except Exception as e:
            print(e)
            continue

Checking subjects:   3%|▎         | 340/11744 [00:12<06:51, 27.74it/s]

114_S_6347_ADNI-T1_2020-09-03_12_45_00.0: failed (removing)


Checking subjects:   3%|▎         | 347/11744 [00:13<11:36, 16.36it/s]

REMOVING 114_S_6347_ADNI-T1_2020-09-03_12_45_00.0


Checking subjects:   4%|▎         | 439/11744 [00:16<06:10, 30.53it/s]

...

REMOVING 003_S_5209_ADNI-T1_2013-07-09_13_32_21.0


Checking subjects: 100%|█████████▉| 11729/11744 [07:24<00:00, 26.50it/s]

130_S_6647_ADNI-T1_2018-12-14_11_22_39.0: no log file (removing)
Not a directory: /rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k/130_S_6647_ADNI-T1_2018-12-14_11_22_39.0


Checking subjects: 100%|██████████| 11744/11744 [07:24<00:00, 26.39it/s]

031_S_0830_ADNI-T1_2011-09-22_15_35_51.0_real: no log file (removing)
Not a directory: /rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k/031_S_0830_ADNI-T1_2011-09-22_15_35_51.0_real


### Collate a list of patients not successfully reconalled, and then resubmit as a recon-all job

In [15]:
%%bash

outdir=/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k
output_file=/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/successful_scott_10k_reconall_paths.txt
>"$output_file"
cd "$outdir"
find "$outdir" -mindepth 1 -maxdepth 1 -type d >> "$output_file"
wc -l< "$output_file"

11176


In [16]:
output_file='/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_t1_paths.txt'

t1_codes = {}
with open(output_file) as t1_file:
    file = t1_file.readlines()
    for path in file:
        patient_name_parts = path.split('/')
        patient_name = patient_name_parts[-1]
        patient_name = patient_name.replace('\n','')
        patient_name = patient_name.replace('.nii.gz','')
        patient_name = patient_name.strip()
        t1_codes.update({patient_name:path})

In [17]:
output_file='/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/successful_scott_10k_reconall_paths.txt'

reconall_successes = []
with open(output_file) as reconall_file:
    file = reconall_file.readlines()
    for path in file:
        patient_name_parts = path.split('/')
        patient_name = patient_name_parts[-1]
        patient_name = patient_name.replace('\n','')
        patient_name = patient_name.strip()
        reconall_successes.append(patient_name)
print(len(reconall_successes))

11176


In [18]:
from itertools import islice

d2 = dict(islice(t1_codes.items(), 2))
print(d2)
print(reconall_successes[0])

{'012_S_4849_ADNI-T1_2012-11-05_12_04_44.0': '/rds/general/project/scott_data_adni/live/ADNI/ADNI_NIFTI/ADNI_allT1/012_S_4849/2012-11-05_12_04_44.0/012_S_4849_ADNI-T1_2012-11-05_12_04_44.0.nii.gz\n', '012_S_4849_ADNI-T1_2012-07-24_09_50_58.0': '/rds/general/project/scott_data_adni/live/ADNI/ADNI_NIFTI/ADNI_allT1/012_S_4849/2012-07-24_09_50_58.0/012_S_4849_ADNI-T1_2012-07-24_09_50_58.0.nii.gz\n'}
123_S_0298_ADNI-T1_2008-05-07_09_18_17.0


In [19]:
import os
missing_reconall_list = '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/missing_reconall_list.txt'
counter = 0

!>"$missing_reconall_list"
path_list = []
for t1 in t1_codes.keys():
    if t1 not in reconall_successes:
        path = t1_codes[t1]
        path_list.append(path)
        with open(missing_reconall_list,'a') as outfile:
            path.replace('\n','')
            if os.path.exists(path.replace('\n','')):
                outfile.write(path)

!wc -l< '/rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/missing_reconall_list.txt'

4557


In [20]:
%%file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh

#!/bin/bash
#PBS -l walltime=72:00:00
#PBS -l select=1:ncpus=1:mem=30gb
#PBS -J 1-4600
#PBS -N reconall_scott_10k
#PBS -o /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/
#PBS -e /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/

inpath=`head -n ${PBS_ARRAY_INDEX} /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/missing_reconall_list.txt| tail -n 1`
infile=`basename $inpath .nii.gz`;

echo ${infile}

outdir=/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k
mkdir -p "$outdir"

module load tools/prod
module load FreeSurfer/7.4.1-centos8_x86_64 > /dev/null 2>&1
#module load freesurfer/7.2.0 > /dev/null 2>&1 ## for CX1
module load FSL/6.0.5.1-foss-2021a > /dev/null 2>&1
#module load fsl/6.0.5 > /dev/null 2>&1 ##for CX1
module load MATLAB/2024b > /dev/null 2>&1
#module load matlab/2017a > /dev/null 2>&1 ##for CX1

export JOB_NUM=$(echo ${PBS_JOBID} | cut -f 1 -d '.' | cut -f 1 -d '[')
export NEW_TMPDIR="${EPHEMERAL}/${JOB_NUM}.${PBS_ARRAY_INDEX}"
mkdir -p ${NEW_TMPDIR}
export TMPDIR=${NEW_TMPDIR}
export SUBJECTS_DIR="/rds/general/project/scott_data_adni/live/ADNI/ADNI_NIFTI/ADNI_allT1/"
export FS_LICENSE="${HOME}/license.txt"

cmd="recon-all -all -i ${inpath} -subjid ${infile} -sd ${outdir}"
echo ${cmd}
${cmd}

## Runs every 1000 jobs - will update timestamps of all reconall jobs so they don't expire in ephemeral ##
mod=${PBS_ARRAY_INDEX}%1000
if [[ "$mod" -eq 0 ]]; then
    echo "updating timestamps..."
    cd ${outdir}
    find . -print0 | while IFS= read -r -d '' path
        do touch "$path" ;
    done
fi

Overwriting /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh


In [21]:
%%bash

chmod -Rf 775 /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh
qsub /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/FINAL_SCOTT_10K_RECONALL.sh

1682561[].pbs-7


## Recon-all scraping

In [22]:
%%file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_reconall_scraper.sh
#!/bin/bash
#PBS -l walltime=24:00:00
#PBS -l select=1:ncpus=1:mem=10gb
#PBS -N scrapescott10k_safe
#PBS -o /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/
#PBS -e /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/

module load FreeSurfer/7.4.1-centos8_x86_64
module load FSL/6.0.5.1-foss-2021a
module load MATLAB/2024b

export JOB_NUM=$(echo ${PBS_JOBID} | cut -f 1 -d '.' | cut -f 1 -d '[')
export NEW_TMPDIR="${EPHEMERAL}/${JOB_NUM}.${PBS_ARRAY_INDEX}"
mkdir -p ${NEW_TMPDIR}
export TMPDIR=${NEW_TMPDIR}

export SUBJECTS_DIR="/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k/"
export MINTSCRIPTS="/rds/general/project/c3nl_scott_students/live/data/sankeith/scott_10k_reconall_scraped"
export FS_LICENSE="/rds/general/user/sk4724/home/license.txt"

mkdir -p ${MINTSCRIPTS}

echo "SUBJECTS_DIR: ${SUBJECTS_DIR}"

# Function to safely list subjects that have a given .stats file
get_subjects_with_stats() {
    local statfile="$1"
    find "${SUBJECTS_DIR}" -maxdepth 1 -mindepth 1 -type d \
        -exec test -f "{}/stats/${statfile}" \; -print | xargs -n1 basename
}

# --- Aseg stats tables ---
aseg_subjects=$(get_subjects_with_stats "aseg.stats")
if [ -n "$aseg_subjects" ]; then
    echo "Processing aseg stats for subjects:"
    echo "$aseg_subjects"
    asegstats2table --subjects $aseg_subjects --meas volume --skip --tablefile ${MINTSCRIPTS}/aseg_stats.txt
    asegstats2table --subjects $aseg_subjects --meas volume --skip --statsfile wmparc.stats --all-segs --tablefile ${MINTSCRIPTS}/wmparc_stats.txt
fi

# --- Aparc stats (Desikan) ---
for hemi in lh rh; do
    for meas in volume thickness area meancurv; do
        aparc_subjects=$(get_subjects_with_stats "${hemi}.aparc.stats")
        if [ -n "$aparc_subjects" ]; then
            echo "Processing ${hemi} aparc stats (${meas}) for subjects:"
            echo "$aparc_subjects"
            aparcstats2table --subjects $aparc_subjects --hemi $hemi --meas $meas --skip \
                --tablefile ${MINTSCRIPTS}/aparc_${meas}_${hemi}.txt
        fi
    done
done

# --- Aparc stats (a2009s) ---
for hemi in lh rh; do
    for meas in volume thickness area meancurv; do
        aparc_subjects=$(get_subjects_with_stats "${hemi}.aparc.a2009s.stats")
        if [ -n "$aparc_subjects" ]; then
            echo "Processing ${hemi} aparc.a2009s stats (${meas}) for subjects:"
            echo "$aparc_subjects"
            aparcstats2table --subjects $aparc_subjects --hemi $hemi --parc aparc.a2009s --meas $meas --skip \
                -t ${MINTSCRIPTS}/${hemi}.a2009s.${meas}.txt
        fi
    done
done

# --- BA_exvivo ---
for hemi in lh rh; do
    for meas in volume thickness area meancurv; do
        aparc_subjects=$(get_subjects_with_stats "${hemi}.BA_exvivo.stats")
        if [ -n "$aparc_subjects" ]; then
            echo "Processing ${hemi} BA_exvivo stats (${meas}) for subjects:"
            echo "$aparc_subjects"
            aparcstats2table --subjects $aparc_subjects --hemi $hemi --parc BA_exvivo --meas $meas --skip \
                -t ${MINTSCRIPTS}/${hemi}.BA_exvivo.${meas}.txt
        fi
    done
done

echo "All tables processed successfully."


Overwriting /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_reconall_scraper.sh


In [10]:
%%bash

chmod -Rf 775 /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_reconall_scraper.sh
qsub /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scott_10k_reconall_scraper.sh

1682551.pbs-7


In [14]:
%%file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scrapereconall_thresh.py

import os
import glob
import pandas as pd

subjects_dir = "/rds/general/project/c3nl_scott_students/ephemeral/sankeith/reconall_scott_10k/"
output_file = "/rds/general/project/c3nl_scott_students/live/data/sankeith/scott_10k_reconall_scraped/BA_exvivo_thresh_summary.txt"
hemis = ["lh", "rh"]
parc = "BA_exvivo.thresh.stats"

all_data = {}

for hemi in hemis:
    files = glob.glob(os.path.join(subjects_dir, "*", "stats", f"{hemi}.{parc}"))
    for fpath in files:
        # Extract subject from directory two levels up from the file path
        subject = os.path.basename(os.path.dirname(os.path.dirname(fpath)))
        if subject not in all_data:
            all_data[subject] = {}
        
        with open(fpath, 'r') as f:
            for line in f:
                # Skip comment and empty lines
                if line.startswith("#") or line.strip() == "":
                    continue
                parts = line.strip().split()
                # Only parse lines with at least 10 columns (data lines)
                if len(parts) < 10:
                    continue
                
                # StructName is parts[0]
                label = f"{hemi}_{parts[0]}"
                
                # Parse and store relevant fields from columns:
                # SurfArea = parts[2], ThickAvg = parts[4], MeanCurv = parts[6]
                all_data[subject][f"{label}_thickness"] = float(parts[4])
                all_data[subject][f"{label}_area"] = float(parts[2])
                all_data[subject][f"{label}_meancurv"] = float(parts[6])

# Convert the collected data into a DataFrame, with subject IDs as index
df = pd.DataFrame.from_dict(all_data, orient='index')
df.index.name = "Subject"
df = df.sort_index(axis=1)

# Save the DataFrame to a tab-separated text file
df.to_csv(output_file, sep='\t', na_rep='NA')
print(f"Saved summary table to {output_file}")


Overwriting /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scrapereconall_thresh.py


In [11]:
%%file /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/qsub_scrapereconall_thresh.sh

#!/bin/bash
#PBS -l walltime=00:30:00
#PBS -l select=1:ncpus=1:mem=5gb
#PBS -N scott10k_thresh_scrapereconall
#PBS -o /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/
#PBS -e /rds/general/project/c3nl_scott_students/ephemeral/sankeith/scott_10k_logs/recon_all/

module load Python/3.13.1-GCCcore-14.2.0
source /rds/general/user/sk4724/home/venv_1/bin/activate

python /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scrapereconall_thresh.py

Overwriting /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/qsub_scrapereconall_thresh.sh


In [12]:
%%bash

chmod -Rf 775 /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/scrapereconall_thresh.py
chmod -Rf 775 /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/qsub_scrapereconall_thresh.sh
qsub /rds/general/project/c3nl_scott_students/live/sankeith/scott_10k_housekeeping/qsub_scrapereconall_thresh.sh

1682552.pbs-7
